In [1]:
#director,writer
#actor if I can get to it
# Here I will work on the first recommendation
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3

In [2]:
#file_paths
tn_budgets_file_path = './../Data/Phase_1_Project_Data/tn.movie_budgets.csv.gz'
imdb_file_path = './../Data/Phase_1_Project_Data/im.db'


In [3]:
# compressed file is too large for github, file path for extracted files specified below
#creating the tn_budgets_df data Frame
tn_budgets_df = pd.read_csv(tn_budgets_file_path)

In [4]:
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget'].str.replace(",","")
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget_float'].str[1:]
tn_budgets_df['production_budget_float'] = tn_budgets_df['production_budget_float'].astype(float)

In [5]:
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross'].str.replace(",","")
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross_float'].str[1:]
tn_budgets_df['worldwide_gross_float'] = tn_budgets_df['worldwide_gross_float'].astype(float)

In [6]:
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross'].str.replace(",","")
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross_float'].str[1:]
tn_budgets_df['domestic_gross_float'] = tn_budgets_df['domestic_gross_float'].astype(float)

In [7]:
#delete all rows with zeros
tn_budgets_df = tn_budgets_df[tn_budgets_df['production_budget_float'] != 0]
tn_budgets_df = tn_budgets_df[tn_budgets_df['worldwide_gross_float'] != 0]
tn_budgets_df = tn_budgets_df[tn_budgets_df['domestic_gross_float'] != 0]

#sanity check
#tn_budgets_df.loc[tn_budgets_df['domestic_gross_float'] == 0]
#tn_budgets_df.loc[tn_budgets_df['worldwide_gross_float'] == 0]
#tn_budgets_df.loc[tn_budgets_df['production_budget_float'] == 0]

In [8]:
tn_budgets_df['net_income'] = tn_budgets_df['worldwide_gross_float'] - tn_budgets_df['production_budget_float']

In [9]:
tn_budgets_df = tn_budgets_df.loc[tn_budgets_df['net_income'] != 0]

In [10]:
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,production_budget_float,worldwide_gross_float,domestic_gross_float,net_income
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",425000000.0,2.776345e+09,760507625.0,2.351345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",410600000.0,1.045664e+09,241063875.0,6.350639e+08
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",350000000.0,1.497624e+08,42762350.0,-2.002376e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",330600000.0,1.403014e+09,459005868.0,1.072414e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",317000000.0,1.316722e+09,620181382.0,9.997217e+08


In [11]:
tn_budgets_df = tn_budgets_df.drop(['release_date','id','production_budget','domestic_gross','worldwide_gross'],axis=1)

In [12]:
tn_budgets_df.head()

,movie,production_budget_float,worldwide_gross_float,domestic_gross_float,net_income
0,Avatar,425000000.0,2.776345e+09,760507625.0,2.351345e+09
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,1.045664e+09,241063875.0,6.350639e+08
2,Dark Phoenix,350000000.0,1.497624e+08,42762350.0,-2.002376e+08
3,Avengers: Age of Ultron,330600000.0,1.403014e+09,459005868.0,1.072414e+09
4,Star Wars Ep. VIII: The Last Jedi,317000000.0,1.316722e+09,620181382.0,9.997217e+08


In [13]:
# compressed file is too large for github, file path for extracted files specified below
conn = sqlite3.connect(imdb_file_path) 

imdb_db = pd.read_sql('''
                               SELECT
                               *
                               FROM sqlite_master
                               ''', conn)
imdb_db

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [14]:
movie_basics_table = pd.read_sql('''
                               SELECT
                               movie_id,primary_title
                               FROM movie_basics
                               ''', conn)

In [15]:
movie_basics_table.head()

,movie_id,primary_title
0,tt0063540,Sunghursh
1,tt0066787,One Day Before the Rainy Season
2,tt0069049,The Other Side of the Wind
3,tt0069204,Sabse Bada Sukh
4,tt0100275,The Wandering Soap Opera


In [16]:
persons_table = pd.read_sql('''
                               SELECT
                               person_id,primary_name,death_year
                               FROM persons
                               ''', conn)

In [17]:
persons_table.head()

,person_id,primary_name,death_year
0,nm0061671,Mary Ellen Bauder,NaN
1,nm0061865,Joseph Bauer,NaN
2,nm0062070,Bruce Baum,NaN
3,nm0062195,Axel Baumann,NaN
4,nm0062798,Pete Baxter,NaN


In [18]:
directors_table = pd.read_sql('''
                               SELECT
                               *
                               FROM directors
                               ''', conn)

In [19]:
directors_table.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [20]:
directors_df = pd.merge(directors_table, persons_table,left_on='person_id',right_on='person_id')

In [21]:
directors_df = pd.merge(directors_df,movie_basics_table,left_on='movie_id',right_on='movie_id')

In [22]:
directors_df.head()

,movie_id,person_id,primary_name,death_year,primary_title
0,tt0285252,nm0899854,Tony Vitale,NaN,Life's a Beach
1,tt0462036,nm1940585,Bill Haley,NaN,Steve Phoenix: The Untold Story
2,tt0835418,nm0151540,Jay Chandrasekhar,NaN,The Babymakers
3,tt0835418,nm0151540,Jay Chandrasekhar,NaN,The Babymakers
4,tt0859635,nm0151540,Jay Chandrasekhar,NaN,Super Troopers 2


In [23]:
directors_df = pd.merge(tn_budgets_df,directors_df, how='inner',left_on='movie',right_on='primary_title')

In [24]:
# Here I am dropping all the rows in our dataframe where 'movie' is NAN
directors_df = directors_df.dropna(subset=['movie'])

# Here I am dropping all the duplicates
directors_df = directors_df.drop_duplicates(subset='movie')

#sanity checks
#directors_df['movie'].value_counts()
#directors_df['movie'].isnull().sum()

In [25]:
# Need to ensure that the directors are still alive
# Dropping all rows where death_year != Nan
directors_df = directors_df.loc[directors_df['death_year'].isna()]

In [26]:
print(len(directors_df))
print(directors_df['death_year'].isna().sum())

1976
1976


In [27]:
new_directors = directors_df.groupby(by=['primary_name'],as_index=False).mean()

In [28]:
new_directors.head()

,primary_name,production_budget_float,worldwide_gross_float,domestic_gross_float,net_income,death_year
0,Aaron Alon,2100000.0,1381824.0,881824.0,-718176.0,NaN
1,Aashiq Abu,75000000.0,30626690.0,14010690.0,-44373310.0,NaN
2,Abeer Zeibak Haddad,12000000.0,994790.0,870067.0,-11005210.0,NaN
3,Adam Carolla,1500000.0,113169.0,113169.0,-1386831.0,NaN
4,Adam Ethan Crow,1600000.0,4420000.0,4420000.0,2820000.0,NaN


In [29]:
new_directors = new_directors.sort_values(by=['net_income'],ascending=False)
new_directors = new_directors[:10]
new_directors.head()

,primary_name,production_budget_float,worldwide_gross_float,domestic_gross_float,net_income,death_year
108,Atsushi Wada,425000000.0,2.776345e+09,760507625.0,2.351345e+09,NaN
1111,Ravi Punj,200000000.0,2.208208e+09,659363944.0,2.008208e+09,NaN
703,Joss Whedon,277800000.0,1.460475e+09,541142707.5,1.182675e+09,NaN
215,Chi-kin Kwok,150000000.0,1.272470e+09,400738009.0,1.122470e+09,NaN
93,Anthony Russo,240000000.0,1.300869e+09,448882263.0,1.060869e+09,NaN


In [30]:
fig,ax = plt.subplots(figsize=(17,5))
ax.bar(new_directors['primary_name'],new_directors['net_income'])
ax.set_xlabel('Director')
ax.set_ylabel('Average Movie Net Income')
ax.set_title('Director vs. Average Movie Net Income');